In [1]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext # https://spark.apache.org/docs/1.6.1/sql-programming-guide.html
from os.path import join, abspath
import psycopg2
import os
# pip install psycopg2-binary

# now create connection

In [2]:
connection = psycopg2.connect(
    host = 'c451_db_1',
    database = 'irs990',
    user = 'postgres',
    password = 'postgres1234',
    port='5432'
)

In [3]:
cur = connection.cursor()
cur.execute("""SELECT datname from pg_database""")
rows = cur.fetchall()
print('\nShow me the databases:\n')
for row in rows:
    print('   ', row[0])


Show me the databases:

    postgres
    irs990
    template1
    template0


In [4]:
pd.read_sql_query("SELECT * FROM address_table LIMIT 3", connection)

,ein,object_id,RtrnHdr_TxPrdEndDt,RtrnHdr_TxYr,BsnssNm_BsnssNmLn1Txt,BsnssNm_BsnssNmLn2Txt,BsnssOffcr_PrsnNm,BsnssOffcr_PrsnTtlTxt,BsnssOffcr_PhnNm,BsnssOffcr_EmlAddrssTxt,...,USAddrss_AddrssLn1Txt,USAddrss_AddrssLn2Txt,USAddrss_CtyNm,USAddrss_SttAbbrvtnCd,USAddrss_ZIPCd,FrgnAddrss_AddrssLn1Txt,FrgnAddrss_AddrssLn2Txt,FrgnAddrss_CtyNm,FrgnAddrss_PrvncOrSttNm,FrgnAddrss_CntryCd
0,204039856,201602669349300025,2015-12-31,2015,EQUESTRIAN OUTREACH CENTER,None,LESLIE GOVE,DIRECTOR,5176889105,None,...,9551 SUTFIN ROAD,None,JEROME,MI,492499302,None,None,None,None,None
1,262704533,201602699349200200,2015-12-31,2015,GIVING EVERY CHILD KNOWLEDGE OF THE,OUTDOORS,MORGAN MURRI,PRESIDENT,None,None,...,135 COUNTRY CENTER DR 223,None,PAGOASA SPRINGS,CO,81147,None,None,None,None,None
2,275165623,201602709349200305,2015-12-31,2015,NORTH ALLEGHENY BOYS LACROSSE,BOOSTERS ASSN,JUDY DUGAN,TREASURER,None,None,...,PO BOX 521,None,WEXFORD,PA,15090,None,None,None,None,None


In [4]:
%%time
eztable = pd.read_sql_query("""
SELECT address_table.*,
   "PrsnNm",
   "TtlTxt",
   "CmpnstnAmt" 
   FROM return_EZOffcrDrctrTrstEmpl
	LEFT JOIN address_table ON return_EZOffcrDrctrTrstEmpl.ein = address_table.ein
	AND return_EZOffcrDrctrTrstEmpl.object_id= address_table.object_id;
""", connection)
eztable.to_csv('../data/temp.csv')

CPU times: user 40.4 s, sys: 5.58 s, total: 46 s
Wall time: 1min 52s


In [6]:
os.getcwd()

'/home/jovyan/scripts'

In [5]:
eztable

,ein,object_id,RtrnHdr_TxPrdEndDt,RtrnHdr_TxYr,BsnssNm_BsnssNmLn1Txt,BsnssNm_BsnssNmLn2Txt,BsnssOffcr_PrsnNm,BsnssOffcr_PrsnTtlTxt,BsnssOffcr_PhnNm,BsnssOffcr_EmlAddrssTxt,...,USAddrss_SttAbbrvtnCd,USAddrss_ZIPCd,FrgnAddrss_AddrssLn1Txt,FrgnAddrss_AddrssLn2Txt,FrgnAddrss_CtyNm,FrgnAddrss_PrvncOrSttNm,FrgnAddrss_CntryCd,PrsnNm,TtlTxt,CmpnstnAmt
0,201247243,201622729349200507,2015-12-31,2015.0,ALIAS CHAMBER ENSEMBLE,None,GEORGEANN BURNS,BOARD PRESIDENT,6153971194,None,...,TN,37219,None,None,None,None,None,GEORGEANN BURNS,PRESIDENT,0
1,201247243,201622729349200507,2015-12-31,2015.0,ALIAS CHAMBER ENSEMBLE,None,GEORGEANN BURNS,BOARD PRESIDENT,6153971194,None,...,TN,37219,None,None,None,None,None,JOCELYN BRIDDELL,VICE PRESIDENT,0
2,201247243,201622729349200507,2015-12-31,2015.0,ALIAS CHAMBER ENSEMBLE,None,GEORGEANN BURNS,BOARD PRESIDENT,6153971194,None,...,TN,37219,None,None,None,None,None,CHRIS FARRIS,DIRECTOR,0
3,201247243,201622729349200507,2015-12-31,2015.0,ALIAS CHAMBER ENSEMBLE,None,GEORGEANN BURNS,BOARD PRESIDENT,6153971194,None,...,TN,37219,None,None,None,None,None,WILLIAM KINSEY NORTON END 82015,DIRECTOR,0
4,201247243,201622729349200507,2015-12-31,2015.0,ALIAS CHAMBER ENSEMBLE,None,GEORGEANN BURNS,BOARD PRESIDENT,6153971194,None,...,TN,37219,None,None,None,None,None,TIFFANY PACK START 62015,DIRECTOR,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4698882,None,None,None,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,ELIZABETH MCCULLOUGH,BOARD MEMBER,0
4698883,None,None,None,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,DAVE POLAND,PAST PRESIDENT,0
4698884,None,None,None,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,NICK WALTERS,TREASURER,0
4698885,None,None,None,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,BETH BLACK,EXECUTIVE DIRECTOR,0


In [5]:
# Create the session
conf = (SparkConf()
    .set("spark.ui.port", "4041")
    .set('spark.executor.memory', '4G')
    .set('spark.driver.memory', '45G')
    .set('spark.driver.maxResultSize', '10G')
    .set('spark.jars', '/home/jovyan/scratch/postgresql-42.2.18.jar'))

# Create the context
# sc = pyspark.SparkContext(conf=conf)
# sqlContext = SQLContext(sc)
spark = SparkSession.builder \
    .appName('test') \
    .config(conf=conf) \
    .getOrCreate()

In [6]:
# spark.stop()
properties = {
    'driver': 'org.postgresql.Driver',
    'url': 'jdbc:postgresql://c451_db_1:5432/irs990',
    'user': 'postgres',
    'password': 'postgres1234',
    'dbtable': 'address_table',
}

In [7]:
spark
os.listdir()
os.getcwd()


'/home/jovyan/scripts'

In [8]:
jdbcDF = spark.read \
    .format('jdbc') \
    .option('driver', properties['driver']) \
    .option('url', properties['url']) \
    .option('dbtable', properties['dbtable']) \
    .option('user', properties['user']) \
    .load()

In [9]:
jdbcDF.show(10)

+---------+------------------+------------------+------------+---------------------+---------------------+-----------------+---------------------+----------------+-----------------------+------------------+---------------------+---------------------+---------------+---------------------+--------------+-----------------------+-----------------------+----------------+-----------------------+------------------+
|      ein|         object_id|RtrnHdr_TxPrdEndDt|RtrnHdr_TxYr|BsnssNm_BsnssNmLn1Txt|BsnssNm_BsnssNmLn2Txt|BsnssOffcr_PrsnNm|BsnssOffcr_PrsnTtlTxt|BsnssOffcr_PhnNm|BsnssOffcr_EmlAddrssTxt|BsnssOffcr_SgntrDt|USAddrss_AddrssLn1Txt|USAddrss_AddrssLn2Txt| USAddrss_CtyNm|USAddrss_SttAbbrvtnCd|USAddrss_ZIPCd|FrgnAddrss_AddrssLn1Txt|FrgnAddrss_AddrssLn2Txt|FrgnAddrss_CtyNm|FrgnAddrss_PrvncOrSttNm|FrgnAddrss_CntryCd|
+---------+------------------+------------------+------------+---------------------+---------------------+-----------------+---------------------+----------------+-------------